#  cf wavelet function

In [1]:
# Input : un dataframe ( comme df_train)
# Output :un dataframe de même taille ; chaque canal de couleur contient les détails obtenus 
#lors de la décomposition en ondelettes   : 30 minutes pour 5000 images

In [2]:
%pylab inline
import pandas
import cvxopt
import time
import numpy as np
import numpy.linalg as la
import scipy
from scipy.stats import mode
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


# Importation de données

In [3]:
df_train=pandas.read_csv('/Users/philippe/Documents/MVA/kernel/kaggle/data/Xtr.csv',header=None,sep=',')
df_train=df_train.drop(3072,1)
df_test=pandas.read_csv('/Users/philippe/Documents/MVA/kernel/kaggle/data/Xte.csv',header=None,sep=',')
df_test=df_test.drop(3072,1)
y_train=pandas.read_csv('/Users/philippe/Documents/MVA/kernel/kaggle/data/Ytr.csv',sep=',')
y_train=pandas.DataFrame(y_train['Prediction'])

# code wavelets

In [4]:
def length(x=0):

    import numpy as np
    l = np.max(np.shape(x))
    return l

################# 1D convolution	

def filter_1d(xin=0,h=0,boption=3):

    import numpy as np
    import scipy.linalg as lng
    import copy as cp    
    
    x = np.squeeze(cp.copy(xin));
    n = length(x);
    m = length(h);
    y = cp.copy(x);

    z = np.zeros(1,m);

    for r in range(np.int(np.int(m/2))):
                
        if boption == 1: # --- zero padding
                        
            z = np.concatenate([np.zeros(m-r-np.int(m/2)-1),x[0:r+np.int(m/2)+1]],axis=0)
        
        if boption == 2: # --- periodicity
            
            z = np.concatenate([x[n-(m-(r+np.int(m/2)))+1:n],x[0:r+np.int(m/2)+1]],axis=0)
        
        if boption == 3: # --- mirror
            
            u = x[0:m-(r+np.int(m/2))-1];
            u = u[::-1]
            z = np.concatenate([u,x[0:r+np.int(m/2)+1]],axis=0)
                                     
        y[r] = np.sum(z*h)
        
        
    

    a = np.arange(np.int(np.int(m/2)),np.int(n-m+np.int(m/2)),1)

    for r in a:
        
        y[r] = np.sum(h*x[r-np.int(m/2):m+r-np.int(m/2)])
    

    a = np.arange(np.int(n-m+np.int(m/2)+1),n,1)

    for r in a:
            
        if boption == 1: # --- zero padding
            
            z = np.concatenate([x[r-np.int(m/2):n],np.zeros(m - (n-r) - np.int(m/2))],axis=0)
        
        if boption == 2: # --- periodicity
            
            z = np.concatenate([x[r-np.int(m/2):n],x[0:m - (n-r) - np.int(m/2)]],axis=0)
        
        if boption == 3: # --- mirror
                        
            u = x[n - (m - (n-r) - np.int(m/2) -1)-1:n]
            u = u[::-1]
            z = np.concatenate([x[r-np.int(m/2):n],u],axis=0)
                    
        y[r] = np.sum(z*h)
    	
    return y


In [5]:
################# 1D convolution with the "a trous" algorithm	

def Apply_H1(x=0,h=0,scale=1,boption=3):

	import numpy as np
	import copy as cp
	
	m = length(h)
	
	if scale > 1:
		p = (m-1)*np.power(2,(scale-1)) + 1
		g = np.zeros( p)
		z = np.linspace(0,m-1,m)*np.power(2,(scale-1))
		g[z.astype(int)] = h
	
	else:
		g = h
				
	y = filter_1d(x,g,boption)
	
	return y
	

In [6]:
################# 2D "a trous" algorithm

def Starlet_Forward(x=0,h=[0.0625,0.25,0.375,0.25,0.0625],J=1,boption=3):

	import numpy as np
	import copy as cp
	
	nx = np.shape(x)
	c = np.zeros((nx[0],nx[1]))
	w = np.zeros((nx[0],nx[1],J))

	c = cp.copy(x)
	cnew = cp.copy(x)
	
	for scale in range(J):
		
		for r in range(nx[0]):
			
			cnew[r,:] = Apply_H1(c[r,:],h,scale,boption)
			
		for r in range(nx[1]):
		
			cnew[:,r] = Apply_H1(cnew[:,r],h,scale,boption)
			
		w[:,:,scale] = c - cnew;
		
		c = cp.copy(cnew);

	return c,w
	

In [7]:
    
	################# 2D "a trous" algorithm

def Starlet_Inverse(c=0,w=0):

	import numpy as np
	
	x = c+np.sum(w,axis=2)

	return x

In [8]:
def extract_detail(img,k) :
    boption=2
    h = 1/16*np.array([1,4,6,4,1])
    J = 3
    c,w =Starlet_Forward(img[:,:,k],h,J,boption)
    return w[:,:,0].flatten()

In [9]:
def return_base(img):
    res=np.empty([3,1024])
    for k in range(3):
        res[k,:]=extract_detail(img,k)
    return res.flatten()    

In [60]:
def wavelet(df):
    n_sample=df.shape[0]
    df_bis=np.empty(df.shape)
    print(n_sample)
    for nb_im in range(n_sample):
        if nb_im%50==0:
            print ("image num" ,nb_im )
        img=df.iloc[nb_im,:].values.reshape(3,32,32).transpose(1,2,0)
        df_bis[nb_im,:]=return_base(img)
    return pandas.DataFrame(df_bis)  

# Transformation

In [46]:
Xtr=pandas.read_csv('/Users/philippe/Documents/MVA/kernel/kaggle/data/Xtr.csv',header=None,sep=',')
Xtr=Xtr.drop(3072,1)
Xte=pandas.read_csv('/Users/philippe/Documents/MVA/kernel/kaggle/data/Xte.csv',header=None,sep=',')
Xte=Xte.drop(3072,1)

In [54]:
Xte.shape


(2000, 3072)

In [43]:
img=df_train.iloc[nb_im,:].values.reshape(3,32,32).transpose(1,2,0)

In [32]:
import pickle

In [61]:
import time
start = time.time()
Xte_w=wavelet(Xte)
end = time.time()
print(end - start)  


output = open('Xte_v.pkl', 'wb')
pickle.dump(df_train, output)
output.close()

2000
image num 0
image num 50
image num 100
image num 150
image num 200
image num 250
image num 300
image num 350
image num 400
image num 450
image num 500
image num 550
image num 600
image num 650
image num 700
image num 750
image num 800
image num 850
image num 900
image num 950
image num 1000
image num 1050
image num 1100
image num 1150
image num 1200
image num 1250
image num 1300
image num 1350
image num 1400
image num 1450
image num 1500
image num 1550
image num 1600
image num 1650
image num 1700
image num 1750
image num 1800
image num 1850
image num 1900
image num 1950
948.694342136383


In [62]:
start = time.time()
Xtr_w=wavelet(Xtr)
output = open('Xtr_v.pkl', 'wb')
end = time.time()
print(end - start)  

pickle.dump(df_train, output)
output.close()

5000
image num 0
image num 50
image num 100
image num 150
image num 200
image num 250
image num 300
image num 350
image num 400
image num 450
image num 500
image num 550
image num 600
image num 650
image num 700
image num 750
image num 800
image num 850
image num 900
image num 950
image num 1000
image num 1050
image num 1100
image num 1150
image num 1200
image num 1250
image num 1300
image num 1350
image num 1400
image num 1450
image num 1500
image num 1550
image num 1600
image num 1650
image num 1700
image num 1750
image num 1800
image num 1850
image num 1900
image num 1950
image num 2000
image num 2050
image num 2100
image num 2150
image num 2200
image num 2250
image num 2300
image num 2350
image num 2400
image num 2450
image num 2500
image num 2550
image num 2600
image num 2650
image num 2700
image num 2750
image num 2800
image num 2850
image num 2900
image num 2950
image num 3000
image num 3050
image num 3100
image num 3150
image num 3200
image num 3250
image num 3300
image num 335

# Load

In [17]:
pkl_file = open('Xtr_v.pkl', 'rb')
Xtr_v= pickle.load(pkl_file)
pkl_file.close()

In [18]:
pkl_file = open('Xte_v.pkl', 'rb')
Xte_v= pickle.load(pkl_file)
pkl_file.close()

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,0.007018,0.000323,0.002215,0.000781,-0.005636,-0.001525,-0.001090,-0.001907,0.004179,-0.004225,...,0.004146,-0.002166,-0.005094,0.001906,-0.006143,-0.013265,-0.013873,0.005223,-0.000860,-0.012881
1,0.000819,0.001688,0.002698,0.004685,0.011166,0.017482,0.045989,0.031377,0.032150,0.062066,...,-0.000530,0.007203,0.008634,0.006800,0.014114,0.000243,-0.019384,-0.046763,-0.048919,-0.057449
2,-0.016779,0.006662,-0.007226,-0.003798,-0.004273,-0.009955,-0.030925,-0.007064,0.008136,0.000618,...,-0.029953,-0.023748,0.047707,0.072310,0.056837,0.045410,0.015561,0.003272,-0.013745,0.000968
3,0.014936,0.004218,0.009732,0.007309,0.004914,0.008172,0.014205,-0.023263,-0.023014,0.011482,...,-0.004505,-0.029634,-0.024069,-0.000788,-0.005010,-0.004260,0.014308,-0.010997,-0.025966,-0.025786
4,-0.073091,-0.046800,-0.056235,-0.063619,-0.088387,-0.044682,-0.014172,-0.077535,-0.100056,-0.066161,...,0.011063,-0.018166,0.012983,0.022676,0.014233,0.047403,0.052239,-0.029272,0.001368,-0.001475
